<a id='toppen'></a>
# Kokebok
#### [Git tips](#git)
[Lenke]

#### [Databehandling](#databehandling)
Viser eksempler for hvordan man kan:
- opprette nye kolonner
- filtrere datasettet
- velge ut kolonner fra en dataframe
- plukke ut en substring og lage ny kolonne med denne verdien
- normalisering
- binning
- opprette dummy variabler
- gruppere og aggregere datasettet
- Endre kolonneverdier

#### [Deskriptiv statistikk](#deskriptiv)
Enkle måter å få oversikt over et datasett
#### [Korrelasjonstester](#korrelasjonstester)
Eksempel på korrelasjonstester man kan gjennomføre og visualiseringer for å gjøre resultatene lettere å lese
#### [Funksjoner](#funksjoner)
Funksjoner som er skrevet på en måte som gjør de lette å gjennbruke i andre programmer. For eksempel kode som oppretter variabel for størrelsesgrupper
#### [Notebook tips](#tips)
Noen tips for å gjøre notebooks lettere å lese

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from scipy import stats
#import statsmodels as stats

In [ ]:
df = px.data.tips() # Testdatasett, finnes flere eksempler: https://plotly.com/python-api-reference/generated/plotly.express.data.html

In [ ]:
df

<a id='databehandling'></a>
## Databehandling

### Filtrering med .loc
Svært nyttig for å filtrere dataframes.

In [ ]:
# Filtrere til ny dataframe
dff = df.loc[df["smoker"] == "Yes"]
dff

In [ ]:
# Filtrere til ny dataframe med flere betingelser
dff = df.loc[(df["smoker"] == "Yes") & (df["sex"] == "Male") & (df["day"] == "Sat")] # Når det er flere conditions må hver av de være i () med & mellom 
dff

### Opprette ny kolonne

In [ ]:
# Opprette en ny kolonne med fast verdi
df["kommunenr"] = "0301"

### Opprette kolonner med .loc
Anbefaler å bruke .loc til å opprette nye kolonner med verdi som avhenger av andre kolonner

In [ ]:
# Endre kolonneverdi basert på andre kolonneverdier
df["gruppe"] = False
df.loc[df["size"] > 2, "gruppe"] = True

In [ ]:
# loc med en liste av aksepterte verdier
df.loc[df["day"].isin(["Sat", "Sun"]), "helg"] = True

In [ ]:
# loc basert på to verdier
df.loc[(df["total_bill"] > 10) & (df["total_bill"] < 20), "bill 10-20"] = 1 # Når det er flere conditions må hver av de være i () med & mellom 

In [ ]:
# Avansert. Printer ut i løkken for å vise hva som skjer
# Koden nedenfor tar utgangspunkt i en kolonne og lager egne kolonner med 0/1 (False/True) verdier for å markere hvorvidt en verdi er innenfor en gitt størrelsesgruppe.
# Om verdiene i dictet endres så kan den generelle koden brukes på andre datasett, uavhengig av hvor mange størrelsesgrupper man definerer. Kan også settes opp som en gjenbrukbar funksjon, se neste celle.

eksempel_dict = {
    "variabelnavn": "total_bill",
    "grupper": {
        "bill 10-19": [10, 19.99],
        "bill 20-29": [20, 29.99],
        "bill 30-39": [30, 30.99]
    }
}

print(eksempel_dict["variabelnavn"])

for i in eksempel_dict["grupper"]:
    print(i)
    print(eksempel_dict["grupper"][i][0])
    print(eksempel_dict["grupper"][i][1])
    df.loc[(df[eksempel_dict["variabelnavn"]] > eksempel_dict["grupper"][i][0]) & (df[eksempel_dict["variabelnavn"]] < eksempel_dict["grupper"][i][1]), i] = 1
df

In [ ]:
# for-løkken fra forrige celle satt inn i en funksjon
# En fordel med å kode funksjoner er at dersom man gjør nøyaktig samme operasjon flere ganger forkorter det koden og den blir mer lesbar

def gruppering(df, dictionary):
    for i in dictionary["grupper"]:
        df.loc[(df[dictionary["variabelnavn"]] > dictionary["grupper"][i][0]) & (df[dictionary["variabelnavn"]] < dictionary["grupper"][i][1]), i] = 1
    return df


eksempel_dict = {
    "variabelnavn": "total_bill",
    "grupper": {
        "bill 10-19": [10, 19.99],
        "bill 20-29": [20, 29.99],
        "bill 30-39": [30, 30.99]
    }
}
dff = gruppering(df, eksempel_dict)
dff

### Velge ut spesifikke kolonner

In [ ]:
# Nedenfor er eksempel på å velge ut kun de kolonnene man ønsker for å forkorte datasett/bare ta med aktuelle kolonner videre. Begge eksemplene oppnår det samme resultatet

# Eksempel 1
dff = df[["sex", "day", "total_bill", "tip"]]

# Eksempel 2
liste = ["sex", "day", "total_bill", "tip"]
dff = df[liste]

dff

### Substring til ny kolonne

In [ ]:
# Hente ut en substring
# Eksempel. Henter ut fylkesnummer fra kommunenummer
df["fylkenr"] = df["kommunenr"].str[:2] # .str gjør verdien om til string og [:2] tar vare på de 2 første tegnene

### Normalisering

In [ ]:
# simple feature scaling
df["total_bill simple feature scaling"] = df["total_bill"]/df["total_bill"].max()

# min-max
df["total_bill min-max"] = (df["total_bill"]-df["total_bill"].min()) / (df["total_bill"].max()-df["total_bill"].min())

# Z-score
df["total_bill z-score"] = (df["total_bill"] - df["total_bill"].mean()) / df["total_bill"].std()

### Binning

In [ ]:
bins = np.linspace(min(df["total_bill"]), max(df["total_bill"]), 4)
group_names = ["Low", "Medium", "High"]
df["total_bill_binned"] = pd.cut(
    df["total_bill"],
    bins,
    labels=group_names,
    include_lowest=True
)

### Dummy variabler

In [ ]:
# pd.get_dummies(df["day"]) # Lager dummy

# Slår sammen dummy variabelen med resten av dataframe
# df = pd.concat([df, pd.get_dummies(df["day"])], axis=1)

# Som funksjon. Best hvis det er flere kolonner det skal lages dummies for
def dummies(df, kolonneliste):
    for i in kolonneliste:
        df = pd.concat([df, pd.get_dummies(df[i])], axis=1)
    return df

df = dummies(df, ["day", "sex", "time"])
df

### Gruppering og aggregering

In [ ]:
# Summere flere kolonner radvis
column_names = ['total_bill', 'tip']
df['bill+tip']= df[column_names].sum(axis=1)

In [ ]:
# Enkel summering med 1 nivå
df.groupby("sex").sum()

In [ ]:
# Enkel summering med 2 nivå
df.groupby(["day", "sex"]).sum()

In [ ]:
# Mer skreddersydd aggregering med egendefinert aggregeringsmetode per kolonne
df.groupby(["day", "sex"]).agg({"total_bill": "mean", "tip": "mean", "size": "count", "bill+tip": "sum"})

# Merk at med .agg metoden får man kun ut de kolonnene som man definerer en metode for

In [ ]:
# Gruppere opp til dagsnivå fra aggregering med to nivåer
eksempel = {
    "total_bill": "sum", 
    "tip": "sum",
    "size": "count", 
    "bill+tip": "sum"
}
dff = df.groupby(["day", "sex"]).agg(eksempel).reset_index()
dff.groupby("day").agg(eksempel)

In [ ]:
# Mer avansert
#####################
# Merk at i eksempelet over så blir size feil om man teller på samme måte begge gangene. Kan derfor lage nested dictionary for at det skal bli mer riktig

hoved_dict = {
    "total_bill": ["sum", "sum"],
    "tip": ["sum", "sum"],
    "size": ["count", "sum"], 
    "bill+tip": ["sum", "sum"]
}

midlertidig_dict = {} # Lager et tomt dict for å putte den midlertidige versjonen av hoved_dict

# Dictionaries består av {key : value} par hvor man kan hente ut verdien ved å bruke key (eksempel nedenfor). Dette brukes i for-løkkene til å lage et midlertidig dict basert på hoved_dictet
print(hoved_dict["tip"])

for i in hoved_dict: # For hver key i hoved_dict
    midlertidig_dict[i] = hoved_dict[i][0] # Setter verdien for hver key

dff = df.groupby(["day", "sex"]).agg(midlertidig_dict).reset_index()
dff
for i in hoved_dict:
    midlertidig_dict[i] = hoved_dict[i][1]

dff.groupby("day").agg(midlertidig_dict)

### Endre kolonneverdier
.replace brukes for å erstatte kolonneverdier med nye verdier. Tar imot f.eks. et dict med {"tidligere navn": "nytt navn",} par

In [ ]:
# Opprette ny kolonne med andre verdier, basert på annen kolonne.
df["dag"] = df["day"].replace({  # man kan også skrive df["day"] = df["day"].replace... for å endre eksisterende kolonne istedenfor å lage en ny
    "Mon": "Mandag",
    "Tue": "Tirsdag",
    "Wed": "Onsdag",
    "Thur": "Torsdag",
    "Fri": "Fredag",
    "Sat": "Lørdag",
    "Sun": "Søndag"
})

# Eksempelet nedenfor er samme kode som det over, men dictionary er definert utenfor koden. Om man har mange verdier kan dette gjøre det lettere å lese scriptet senere.

eksempel_dict = {
    "Mon": "Mandag",
    "Tue": "Tirsdag",
    "Wed": "Onsdag",
    "Thur": "Torsdag",
    "Fri": "Fredag",
    "Sat": "Lørdag",
    "Sun": "Søndag"
}

df["dag"] = df["day"].replace(eksempel_dict)

In [ ]:
# Eksempel med fylkenr til fylkenavn
df["fylke"] = df["fylkenr"].replace({ # Samme logikk for å Lage ny kolonne med fullt fylkesnavn
    '03': '03 Oslo',
    '11': '11 Rogaland',
    '15': '15 Møre og Romsdal',
    '18': '18 Nordland',
    '30': '30 Viken',
    '34': '34 Innlandet',
    '38': '38 Vestfold og Telemark',
    '42': '42 Agder',
    '46': '46 Vestland',
    '50': '50 Trøndelag',
    '54': '54 Troms og Finnmark'
})

### .select
Fra numpy


In [ ]:
# Samme prinsipp som .loc, men med numpy .select
df["I Oslo"] = np.select([df["fylkenr"].eq("03") & df["kommunenr"].eq("0301")], "1", default="0")

<a id='deskriptiv'></a>
## Deskriptiv statistikk

In [ ]:
df.describe(include="all").transpose() # Inkluderer alle for å vise mer enn bare numeriske kolonner
# Merk at .transpose() ikke er nødvendig, men jeg synes det gjør beskrivelsen lettere å lese

In [ ]:
df["day"].value_counts()

In [ ]:
sns.boxplot(x = "day", y = "total_bill", data = df).set_title("Seaborn")

px.box(df, x = "day", y = "total_bill", title = "Plotly")

In [ ]:
px.scatter(df, x = "total_bill", y = "tip")

In [ ]:
sns.regplot(x = "total_bill", y = "tip", data = df)

#### Plotly eksempler

In [ ]:
px.bar(
    df.groupby("sex").sum().reset_index(),  # Dataframen som benyttes til figuren, man kan gjøre gruppering o.l. på den i samme linje
    x = "sex",
    y = "total_bill"
)

<a id='korrelasjonstester'></a>
## Korrelasjonstester

In [ ]:
df = px.data.tips() # Laster inn på nytt siden koden over lager mye støy

In [ ]:
pearson_coef, p_value = stats.pearsonr(df["total_bill"], df["tip"])
pearson_coef, p_value

In [ ]:
df.corr() # Kan visualiseres, se nedenfor

In [ ]:
px.imshow(
    df.corr(),
    color_continuous_scale = "rdbu", # Foretrekker denne colorscalen, zmin/max på -1 og 1 gjør fargen mer intuitiv å lese
    zmin = -1,
    zmax = 1
) # Skal kunne legge inn text_auto = True for å få tall i rutene, men fungerer ikke. Muligens for gammel versjon av plotly installert

In [ ]:
sns.heatmap(df.corr(), xticklabels = df.columns, yticklabels = df.columns, annot = True)

<a id='funksjoner'></a>
## Funksjoner
Mange måter å lage og bruke funksjoner. Det er god praksis at de dokumenteres i koden.
- Standard formateringen av slik dokumentasjon kan finnes her: https://numpydoc.readthedocs.io/en/latest/format.html#docstring-standard


In [ ]:
def prosent(df, a, b):
    """Lager en kolonne for prosent
    Tar utgangspunkt i kolonne a og regner ut prosenten den utgjør av kolonne b
    
    Parameters
    ----------
    a
        Kolonnen som inneholder deltall
    b
        Kolonnen som inneholder heltall
    
    Returns
    -------
    Dataframe med en ny kolonne som har utregnet prosent basert på kolonne a og b
    """
    df["Prosent"] = df[a]/df[b]
    return df

In [ ]:
help(prosent)

In [ ]:
df = prosent(df, "tip", "total_bill")
df

### lambda

In [ ]:
a = lambda x: print(str(x) +" er et partall") if (x % 2) == 0 else print(str(x) +" er et oddetall")
a(2)

In [ ]:
# Oppretter ny kolone med lambda
df.assign(Prosent = lambda x: x["tip"] / x["total_bill"])

In [ ]:
# Oppretter ny kolonne basert på betingelser med lambda
df["kjønn_kode"] = df.apply(lambda x: "M" if x.sex != "Female" else "F", axis = 1)

### Ferdige funksjoner

In [ ]:
# Sortering av driftsstørrelser, gjør at man kan summere kolonnen og få vite hvor mange som har den driftsstørrelsen

def gruppering(df, dictionary, kolonnenavn = None):
    if kolonnenavn == None:
        for i in dictionary["grupper"]:
            df.loc[(df[dictionary["variabelnavn"]] > dictionary["grupper"][i][0]) & (df[dictionary["variabelnavn"]] < dictionary["grupper"][i][1]), i] = 1
        return df
    else:
        for i in dictionary["grupper"]:
            df.loc[(df[dictionary["variabelnavn"]] > dictionary["grupper"][i][0]) & (df[dictionary["variabelnavn"]] < dictionary["grupper"][i][1]), kolonnenavn] = i
        return df


størrelsesgrupper = {
    "variabelnavn": "drift",  # Endre denne til navn på areal i drift variabelen
    "grupper": {
        "0 dekar": [float('-inf'), 0.99],
        "1 - 4 dekar": [1, 4.99],
        "5 - 49 dekar": [5, 49.99],
        "50 - 99 dekar": [50, 99.99],
        "100 - 199 dekar": [100, 199.99],
        "200 - 299 dekar": [200, 299.99],
        "300 - 499 dekar": [300, 499.99],
        "500 dekar og mer": [500, float('inf')]
    }
}
df = gruppering(df, størrelsesgrupper) # Lager 1 kolonne per gruppe med 1 som verdi i gruppen raden tilhører
df = gruppering(df, størrelsesgrupper, "størrelsesgruppe - drift") # Lager 1 kolonne for alle gruppene med merkelappen som verdi i kolonnen

In [ ]:
""" For å sammenligne to dataframes kan man bruke funksjonen nedenfor. 
"""
def sammenligning(a, b):
    liste = df.columns.drop(set(a.columns) - set(b.columns))
    print("Sammenligner kolonner i dataframes")
    for i in a.columns:
        if i not in liste:
            print(str(i) + " er ikke i a")
    for i in b.columns:
        if i not in liste:
            print(str(i) + " er ikke i b")
    print()
    print("Differanser i kolonneverdi for delte kolonner")
    count = 0
    for i in liste:
        if df[i].dtype == float:
            if df[i].sum() - dff[i].sum() != 0:
                print(str(i))
                print(df[i].sum() - dff[i].sum())
                count = count+1
    print("Forskjellig kolonneverdi i " + str(count) + " kolonner")

sammenligning(df, df_du_skal_sammenligne)

<a id='tips'></a>

## Notebook tips

Om du se datasettet i notebooken er det praktisk å bruke .head() eller .tail() for å unngå at tabellene blir for lange.

Bruk git knappen aktivt.
Hvis du jevnlig kommenterer tanken bak endringene du gjør via git knappen og lagrer de med "COMMIT"-knappen blir det generert veldig mye dokumentasjon. Det blir også lett å finne tilbake til versjoner av programmet som fungerte om noe skulle gå galt.

Om du er usikker på hva du har endret siden sist kan du bruke git knappen øverst eller gå på sidepanelet for git og klikke på "History" for å se tidligere versjoner av notebooken sammen med tilhørende kommentar

Du kan lage klikkbare lenker i notebooks.
For å lage en lenke legger du inn denne koden i en markdown celle:
~~~
[To some Internal Section](#section_id)
~~~
Og denne koden der du vil at lenken skal føre:
~~~
<a id='section_id'></a>
~~~

Dobbeltklikk på denne cellen for å se et eksempel

## [Til toppen](#toppen)